In [ ]:
!pip install ipywidgets

In [1]:
import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [2]:
!pip uninstall torch torchvision torchaudio -y
!pip cache purge
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip show torch | findstr Version


Found existing installation: torch 2.3.1
Uninstalling torch-2.3.1:
  Successfully uninstalled torch-2.3.1
Found existing installation: torchvision 0.18.1
Uninstalling torchvision-0.18.1:
  Successfully uninstalled torchvision-0.18.1
Found existing installation: torchaudio 2.3.1
Uninstalling torchaudio-2.3.1:
  Successfully uninstalled torchaudio-2.3.1


You can safely remove it manually.


Files removed: 1542 (969.2 MB)
Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/2.4 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.4 GB 95.6 MB/s eta 0:00:26
      --------------------------------------- 0.0/2.4 GB 100.7 MB/s eta 0:00:24
      --------------------------------------- 0.1/2.4 GB 97.7 MB/s eta 0:00:25
     - -------------------------------------- 0.1/2.4 GB 94.3 MB/s eta 0:00:26
     - -------------------------------------- 0.1/2.4 GB 93.8 MB/s eta 0:00:26
     - -------------------------------------- 0.1/2.4 GB 95.0 MB/s eta 0:00:25
     -- ------------------------------------- 0.1/2.4 GB 94.9 MB/s eta 0:00:25
     -- ------------------------------------- 0.2/2.4 GB 94.1 MB/s eta 0:00:25
     -- ------------------------------------- 0.2/2.4 GB 95.4 MB/s eta 0:00:24
     --- ------------------------------------ 0.2/2.4 GB 96.0 MB/s eta 0:00:24
     --- ------------------------------------ 0.

In [1]:
import torch
print(torch.cuda.is_available())  # True면 GPU 사용 가능
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))


True
0
NVIDIA GeForce RTX 3050


# 🎙️ Pyannote Speaker Diarization 코드 설명

이 코드는 Hugging Face의 **pyannote.audio** 라이브러리를 사용하여  
오디오 파일(`audio.wav`)에서 **화자 분리(Speaker Diarization)**를 수행하는 예제입니다.  

---

## 주요 흐름

1. **환경 변수 불러오기**
   - `.env` 파일에 저장된 `HUGGINGFACE_ACCESS_TOKEN`을 불러와서 Hugging Face 모델 접근에 사용합니다.

2. **Pipeline 로드**
   - `pyannote/speaker-diarization-3.1` 모델을 불러옵니다.
   - 최신 버전의 `pyannote.audio`(3.x 계열)에서는 `plda` 같은 파라미터는 필요하지 않습니다.
   - GPU(CUDA)가 가능하다면 자동으로 GPU에서 실행됩니다.

3. **오디오 파일 경로 설정**
   - `os.getcwd()`를 사용해 현재 작업 디렉토리에서 `audio.wav` 파일을 불러옵니다.

4. **Diarization 실행**
   - `pipeline(audio_file, hook=hook)`을 실행하면 화자 분리 결과(`Annotation` 객체)가 반환됩니다.
   - `ProgressHook`을 사용하면 진행 상황이 표시됩니다.

5. **결과 출력**
   - 결과는 `output.itertracks(yield_label=True)`로 순회할 수 있습니다.
   - 여기서 각 변수의 의미는 다음과 같습니다:

---

## 출력 구조 설명

```python
for turn, _, speaker in output.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")
```

-turn

   -Segment 객체 (시작/끝 시간 정보를 담음)

   -예: Segment(0.2, 1.5)

   -접근: turn.start, turn.end

_ (track id)

   -동일 speaker 안에서 여러 트랙을 구분할 때 쓰이는 ID

   -특별히 사용하지 않으므로 _로 무시

-speaker

   -화자 레이블 (문자열)

   -예: "SPEAKER_00", "SPEAKER_01"

# 예시 출력

start=0.2s stop=1.5s speaker_SPEAKER_00

start=1.8s stop=3.9s speaker_SPEAKER_01

start=4.2s stop=5.7s speaker_SPEAKER_00


즉, 오디오의 각 구간별로 시작시간 / 종료시간 / 화자ID가 표시됩니다.

In [7]:
import os
# 현재 실행 중인 코드 파일 기준 절대경로
#오디오 파일 이름
AUDIO_PATH = "data/audio.wav"
BASE_DIR = os.getcwd()
# 
audio_file = os.path.join(BASE_DIR, AUDIO_PATH)

In [8]:
import torch
from pyannote.audio import Pipeline
from pyannote.audio.pipelines.utils.hook import ProgressHook
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()
HF_TOKEN = os.getenv("HUGGINGFACE_ACCESS_TOKEN")
#print("HF_TOKEN:", HF_TOKEN)

# Community-1 open-source speaker diarization pipeline
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=HF_TOKEN
)

# CUDA가 있으면 GPU, 없으면 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline.to(device)

# 실행
with ProgressHook() as hook:
    output = pipeline(audio_file, hook=hook)

# 결과 출력
for turn, _, speaker in output.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")


Output()

c:\Users\SBA\github\test_syh\sesacproject\Lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: 
UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input
numel divided by output numel). (Triggered internally at ..\aten\src\ATen\native\ReduceOps.cpp:1807.)
  std = sequences.std(dim=-1, correction=1)

start=8.7s stop=11.9s speaker_SPEAKER_03
start=16.1s stop=40.1s speaker_SPEAKER_03
start=40.8s stop=48.4s speaker_SPEAKER_00
start=48.6s stop=56.5s speaker_SPEAKER_00
start=57.0s stop=61.6s speaker_SPEAKER_00
start=62.6s stop=71.6s speaker_SPEAKER_01
start=72.2s stop=75.7s speaker_SPEAKER_01
start=75.9s stop=80.6s speaker_SPEAKER_01
start=81.2s stop=85.6s speaker_SPEAKER_01
start=86.6s stop=99.3s speaker_SPEAKER_02
start=99.9s stop=110.0s speaker_SPEAKER_02
start=111.6s stop=118.3s speaker_SPEAKER_03
start=118.9s stop=126.8s speaker_SPEAKER_03
start=129.2s stop=137.0s speaker_SPEAKER_03
start=137.4s stop=141.1s speaker_SPEAKER_03
start=142.1s stop=150.6s speaker_SPEAKER_00
start=151.3s stop=155.7s speaker_SPEAKER_01
start=156.2s stop=160.4s speaker_SPEAKER_01
start=161.4s stop=165.9s speaker_SPEAKER_01
start=167.3s stop=176.0s speaker_SPEAKER_02
start=176.2s stop=184.3s speaker_SPEAKER_02
start=186.0s stop=213.0s speaker_SPEAKER_03
start=214.4s stop=215.1s speaker_SPEAKER_03
start=215.

In [ ]:
import os
from pyannote.audio import Pipeline
from dotenv import load_dotenv

# ==============================
# 1. 환경 변수 로드
# ==============================
load_dotenv()
PYANNOTE_TOKEN = os.getenv("PYANNOTEAI_API_KEY")

if not PYANNOTE_TOKEN:
    raise ValueError("환경 변수에 PYANNOTEAI_API_KEY가 없습니다. .env 파일을 확인하세요.")

# ==============================
# 2. Precision-2 Speaker Diarization (서버 실행)
# ==============================
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-precision-2",
    token=PYANNOTE_TOKEN
)

# ==============================
# 3. 오디오 파일 경로
# ==============================
audio_file = os.path.join("data", "tiro_Test1_4People.wav")

# ==============================
# 4. 실행 (서버에서 처리)
# ==============================
output = pipeline(audio_file)  # pyannoteAI 서버에서 실행됨

# ==============================
# 5. 결과 출력
# ==============================
for turn, speaker in output.speaker_diarization:
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s {speaker}")
